In [123]:
import pandas as pd

df = pd.read_csv("../data/spotify-tracks-audio-features.csv")
# df = df[["acousticness", "energy", "loudness"]]
dataset = df.values
m, n = dataset.shape
x, y = dataset[:,:n-1], dataset[:,n-1]
print(x.shape, y.shape, dataset.shape)

(18750, 13) (18750,) (18750, 14)


In [124]:
import numpy as np

column_mins = []
column_maxs = []

for i in range(n-1):
    column_mins.append(min(x[:,i]))
    column_maxs.append(max(x[:,i]))

x_norm = np.zeros((m, n-1))

for i in range(m):
    for j in range(n-1):
        xi = x[i, j]
        x_norm[i, j] = (xi - column_mins[j]) / (column_maxs[j] - column_mins[j])

x = x_norm.copy()
print(x)

[[4.30722892e-04 3.78968895e-01 2.53616830e-01 ... 3.10480095e-01
  7.50000000e-01 8.65832213e-02]
 [9.98995984e-01 6.11451943e-01 1.11728124e-01 ... 5.59057899e-01
  0.00000000e+00 3.59677253e-01]
 [2.97188755e-01 5.12431385e-01 1.25194816e-01 ... 4.35938328e-01
  7.50000000e-01 1.69338988e-01]
 ...
 [1.72690763e-01 7.33075019e-01 1.56649013e-01 ... 8.00127369e-01
  7.50000000e-01 2.83128168e-01]
 [9.14658635e-01 2.06759229e-01 1.77596641e-01 ... 4.14579977e-01
  7.50000000e-01 1.38615910e-02]
 [2.74096386e-06 3.85426757e-01 1.31631214e-01 ... 5.85805340e-01
  7.50000000e-01 1.05203269e-01]]


In [125]:
classes = []

for yi in y:
    if yi not in classes:
        classes.append(yi)

classes.sort()
classes

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [126]:
classes_len = len(classes)

y_one_hot = np.zeros((m, classes_len))

for i in range(m):
    yi = int(y[i])
    y_one_hot[i, yi] = 1

y = y_one_hot


In [127]:
train_percentage = 80

train_size = int(m*(train_percentage/100))

x_train, x_test = x[:train_size, :], x[train_size:, :]
y_train, y_test = y[:train_size], y[train_size:]


In [128]:
from sklearn.model_selection import train_test_split

test_size = 1 - (train_percentage/100)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)


In [129]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(24, input_dim=n-1, activation='relu'))
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(20, activation='relu'))
model.add(Dense(len(classes), activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [130]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
469/469 [==============================] - 1s 1ms/step - loss: 0.3138 - accuracy: 0.2363
Epoch 2/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2587 - accuracy: 0.3009
Epoch 3/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2454 - accuracy: 0.3237
Epoch 4/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2376 - accuracy: 0.3625
Epoch 5/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2311 - accuracy: 0.3861
Epoch 6/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2276 - accuracy: 0.3985
Epoch 7/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2258 - accuracy: 0.4014
Epoch 8/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2247 - accuracy: 0.4045
Epoch 9/100
469/469 [==============================] - 1s 1ms/step - loss: 0.2239 - accuracy: 0.4016: 0s - loss: 0.2243 - accuracy: 0.
Epoch 10/100
469/469 [==============================] - 1

In [131]:
model.evaluate(x_test, y_test)

118/118 [==============================] - 0s 779us/step - loss: 0.2147 - accuracy: 0.4259


[0.214652881026268, 0.42586666345596313]